In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('./ms_annual_data_2022.csv')

# filter data
data = data.dropna()
# have to reset the index after dropping the null rows or else next cell will error
data = data.reset_index(drop=True)

C:\Users\raims\AppData\Local\Temp\ipykernel_20492\3142806962.py:1: DtypeWarning: Columns (7,13,14,18,20,26,27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./ms_annual_data_2022.csv')


In [3]:
def clean_monetary_value(data):
    if isinstance(data, str):
        cleaned_data = data.replace('$', '').strip()
        cleaned_data = clean_string_with_comma(cleaned_data)
        return float(cleaned_data)
    return data
    
def clean_string_with_comma(data):
    if isinstance(data, str):
        cleaned_data = data.replace(',', '')
        return float(cleaned_data)
    return data

def clean_percentage(data):
    if isinstance(data, str):
        return float(data.strip('%'))
    return data